In [54]:
import create_data
import kmeans
import voltage
import os
import importlib
import time
import bpf
import numpy as np


from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

In [55]:
importlib.reload(create_data)
importlib.reload(kmeans)
importlib.reload(voltage)
importlib.reload(bpf)

<module 'bpf' from 'C:\\Users\\avigh\\Documents\\python\\VoltageDimentionalReduction\\code\\bpf.py'>

In [56]:
print("Loading Data...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(np.int64)

# data = create_data.Data(np.array(X))

Loading Data...


In [57]:
subDivision = {}
summation = {}
count = {}

print("Sorting and averaging...")

for xi, yi in zip(X, y):
    if yi in summation:
        subDivision[yi].append(np.array(xi))
        summation[yi] += np.array(xi)
        count[yi] += 1
    else:
        subDivision[yi] = [np.array(xi)]
        summation[yi] = np.array(xi)
        count[yi] = 1

Sorting and averaging...


In [58]:
print("Kmeans...")

k = 10
landmarks = []
data = []

for yi in range(10):
    print(yi)
    
    partitions = kmeans.Partitions(subDivision[yi])
    partitions.k_means(k, seed=time.time())

    data += list(partitions.centers)

for yi in range(10):
    landmarks.append(voltage.createLandmarkClosestTo(data, summation[yi] / count[yi], 1))

data = create_data.Data(data)
print(len(data))
print([l.index for l in landmarks])

Kmeans...
0
1
2
3
4
5
6
7
8
9
100
[2, 10, 29, 33, 41, 57, 65, 79, 86, 92]


In [59]:
print(type(data))
print(isinstance(data, create_data.Data))

<class 'create_data.Data'>
True


In [65]:
print("Parameter Finding...")

cs = []
pgs = []

for landmark in landmarks:
    c, p_g = bpf.bestParameterFinder(voltage.gaussiankernel, [landmark], data, minBound=-10, maxBound=10, granularity=5, epsilon=0.5)
    print(c, p_g)
    cs.append(c)
    pgs.append(p_g)

print(cs)
print(pgs)

Parameter Finding...
10.0 1.0 1.0
1.0 403.4287934927351 0.1353352832366127
5.8 -2.9
10.0 1.0 1.0
1.0 403.4287934927351 1.0
5.8 -0.8
10.0 1.0 1.0
1.0 403.4287934927351 1.0
5.8 -0.7
10.0 1.0 1.0
1.0 403.4287934927351 1.0
5.6 -0.9
10.0 1.0 1.0
1.0 148.4131591025766 0.00012340980408667956
5.0 -8.7
10.0 1.0 1.0
1.0 148.4131591025766 0.0009118819655545162
5.0 -6.6
10.0 1.0 1.0
1.0 403.4287934927351 0.36787944117144233
5.8 -1.4
10.0 1.0 1.0
1.0 148.4131591025766 0.00033546262790251185
5.0 -7.6
10.0 1.0 1.0
1.0 148.4131591025766 0.0009118819655545162
5.0 -6.9
10.0 1.0 1.0
1.0 148.4131591025766 0.0009118819655545162
5.0 -7.1
[5.8, 5.8, 5.8, 5.6, 5.0, 5.0, 5.8, 5.0, 5.0, 5.0]
[-2.9, -0.8, -0.7, -0.9, -8.7, -6.6, -1.4, -7.6, -6.9, -7.1]


In [81]:
c, p_g = bpf.bestParameterFinder(voltage.gaussiankernel, landmarks, data, minBound=-10, maxBound=10, granularity=10, epsilon=0.5)

10.0 1.0 1.0


C:\Users\avigh\Documents\python\VoltageDimentionalReduction\code\voltage.py:53: RuntimeWarning: invalid value encountered in divide
  self.weights = self.weights / self.weights.sum(axis=1, keepdims=True)


1.0 148.4131591025766 7.38905609893065


In [82]:
voltages = []

for index in range(0, len(landmarks)):
    voltages.append(bpf.calculateFor(voltage.gaussiankernel, [landmarks[index]], data, None, c, p_g)[0])

In [83]:
bpf.visualizations(voltages, "../inputoutput/matplotfigures/MNIST")

<Figure size 640x480 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [84]:
print(voltages[0])

[1.89931330e-01 1.41655076e-01 1.00000000e+00 8.27891641e-01
 8.67740070e-01 6.94505768e-01 1.45629941e-01 6.07102223e-01
 2.40813950e-01 7.63390711e-01 2.55671692e-09 1.48185923e-09
 3.05199456e-09 1.79109725e-09 3.28722132e-09 1.67210667e-09
 3.39670701e-09 3.40720928e-09 1.48781318e-08 1.58817278e-09
 6.57833598e-06 3.92638507e-06 4.14877839e-06 5.72386879e-06
 1.07720842e-05 6.04672348e-06 6.49354557e-06 3.47638131e-06
 4.19663649e-06 6.99051847e-06 1.29824735e-05 1.32777141e-05
 5.47508924e-05 4.13686679e-05 3.95782889e-05 1.21179518e-05
 6.25843921e-05 1.02860323e-04 3.87435989e-05 1.38568347e-05
 2.51998302e-07 2.66851718e-07 5.03930103e-07 3.21919131e-07
 5.17487053e-08 3.77974966e-07 5.67398690e-07 4.16443148e-07
 4.29323932e-07 2.19471172e-06 1.39741046e-05 6.26429191e-05
 4.54296285e-05 3.04718359e-04 4.30558585e-05 7.56695684e-05
 6.89313593e-05 3.61598139e-05 2.49835456e-05 3.39427584e-05
 2.42033122e-04 4.00985249e-05 7.95132126e-05 9.82604232e-05
 1.18874736e-04 1.185322

In [85]:
print(voltages[1])

[6.41017011e-06 5.13058521e-06 3.03656868e-05 2.57538556e-05
 3.90988861e-05 9.05760007e-05 5.19074212e-06 1.36885163e-04
 7.95198481e-06 2.47161851e-05 1.00000000e+00 5.58917706e-01
 6.49322300e-01 6.85156186e-01 4.60845073e-01 6.35345901e-01
 4.26041530e-01 5.44346649e-01 5.27350838e-01 6.04560581e-01
 7.58182369e-04 1.64673740e-04 1.80547768e-04 5.68889808e-04
 3.59924489e-04 8.73923275e-04 1.05537338e-03 1.67541683e-04
 1.70635881e-04 7.89816854e-04 3.04658588e-04 7.55628718e-04
 3.50997921e-04 4.70449337e-04 1.99745485e-04 5.36744867e-04
 3.50693868e-04 3.42929537e-04 1.67875541e-04 4.78219368e-04
 1.80472449e-05 1.94143190e-05 1.24704595e-05 1.94553987e-05
 1.34071561e-06 4.59988650e-05 1.31106175e-05 2.44385144e-05
 1.18105448e-05 1.46897579e-05 2.69058012e-04 2.88740851e-04
 4.84667400e-04 5.71841596e-04 1.62612646e-04 2.66246691e-04
 2.57021087e-04 7.12622562e-04 4.80531561e-04 2.91836857e-04
 3.79228384e-05 1.08955915e-05 2.17515156e-05 2.71166639e-05
 4.17003807e-05 3.332724

In [86]:
predicted = np.argmax(voltages, axis=0)
correct = np.repeat(np.arange(10), k)

num_incorrect = np.sum(predicted != correct)

accuracy = np.mean(predicted == correct)
error_rate = 1 - accuracy

print(predicted)
print(f"Incorrect predictions: {num_incorrect}/100")
print(f"Accuracy: {accuracy:.2%}")
print(f"Error Rate: {error_rate:.2%}")

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 9 2 2 5 5 3 3 3 5 3
 3 3 5 4 4 9 4 9 9 9 4 9 9 5 5 5 5 3 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 4
 9 7 4 7 9 7 5 8 8 8 8 5 8 8 8 5 9 9 9 4 4 9 4 9 9 4]
Incorrect predictions: 23/100
Accuracy: 77.00%
Error Rate: 23.00%
